In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/mati/Desktop/set_datos_propiedades.csv')

In [4]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [5]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Random Forests

In [21]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [22]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [24]:
randomforest = RandomForestRegressor()
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 87.80 % , error = 16837007953.427162


## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [ ]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [14]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [ ]:
lista_arboles = [5,10,15,20,30,50,100,500]
lista_feature = [0.3,0.5,0.6,0.7,0.8,1.0]

for cant_arbol in lista_arboles:
    for feature in lista_feature:
        randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
        randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
        precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
        res.append((cant_arbol,feature,precision,error))
        print(cant_arbol,' - ',feature)

In [21]:
for r in res:
    print("Cant arboles = {}, % features = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

Cant arboles = 5, % features = 0.5, error = 15.51140013061924, cantidad = 14859, precision = 85.35
Cant arboles = 5, % features = 0.7, error = 15.50905961189723, cantidad = 14854, precision = 64.52
Cant arboles = 5, % features = 1.0, error = 15.371427728972733, cantidad = 14847, precision = 63.64
Cant arboles = 10, % features = 0.5, error = 15.491710897184387, cantidad = 14859, precision = 85.61
Cant arboles = 10, % features = 0.7, error = 15.470925951849924, cantidad = 14858, precision = 80.35
Cant arboles = 10, % features = 1.0, error = 15.421936805593825, cantidad = 14854, precision = 85.20
Cant arboles = 20, % features = 0.5, error = 15.391098003460806, cantidad = 14857, precision = 80.54
Cant arboles = 20, % features = 0.7, error = 15.429889570567612, cantidad = 14851, precision = 70.31
Cant arboles = 20, % features = 1.0, error = 15.381341070970887, cantidad = 14853, precision = 60.37
Cant arboles = 30, % features = 0.5, error = 15.451073242401757, cantidad = 14858, precision = 8

## Tomamos features = {} y  cant = {}

In [ ]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [41]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [ ]:
cant_arbol = 0
feature = 0

In [42]:
randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)
precision = randomforest.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

86.0141632861


# calculo con los verdaderos datos a analizar

In [25]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [45]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: randomforest.predict(x)[0],axis = 1)

In [26]:
resultado = analizar.loc[:,['id','price_usd']]

In [ ]:
resultado.to_csv('Random_Forests_resultados', index = False)